# CASE STUDY - performance monitoring (solution)


In [11]:
import os
import sys
import csv
import requests
from collections import Counter
from datetime import date
from ast import literal_eval
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

## Getting started


## TASK 1: Write units test for a logger and a logging API endpoint


In [4]:
pip install joblib

     |████████████████████████████████| 307kB 8.2MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [17]:
!python ./unittests/LoggerTests.py

....
----------------------------------------------------------------------
Ran 4 tests in 0.068s

OK


## TASK 2: Add an API endpoint for logging


In [18]:
!python ./unittests/ApiTests.py

ssss
----------------------------------------------------------------------
Ran 4 tests in 0.000s

OK (skipped=4)


In [31]:
## TASK 3: Make sure all tests pass


In [21]:
!python run-tests.py

ssss....... grid searching
... saving test version of model
...
----------------------------------------------------------------------
Ran 11 tests in 37.119s

OK (skipped=4)


In [32]:
## TASK 4: Create model performance investigative tooling


In [22]:
## YOUR CODE HERE

from monitoring import get_latest_train_data, get_monitoring_tools

## load latest data
data = get_latest_train_data()
y = data['y']
X = data['X']

In [23]:
## generate some data
import numpy as np
import pandas as pd
bs_samples = 60
subset_indices = np.random.choice(np.arange(X.shape[0]),
                                  bs_samples,replace=True).astype(int)
mask = np.in1d(np.arange(X.shape[0]),subset_indices)
X_bs=X[mask]
X_outliers = X[:5].copy()
X_outliers['age'] = [88,90,76,80,68]
X_outliers['num_streams'] = [111,100,80,90,150]
X_query = pd.concat([X_bs,X_outliers])

print(X_query.shape)

(63, 4)


In [24]:
pm_tools = get_monitoring_tools(X,y)

In [25]:
## read in the logged data
df = pd.read_csv(logfile)
df.drop(columns=["unique_id","y_proba"], inplace=True)
df.head()

,timestamp,y_pred,query,model_version,runtime
0,1.602822e+09,0.0,"['united_states', 24, 'aavail_basic', 8]",0.1,000:00:00
1,1.602822e+09,1.0,"['singapore', 42, 'aavail_premium', 17]",0.1,000:00:00
2,1.602822e+09,0.0,"['united_states', 20, 'aavail_basic', 14]",0.1,000:00:00


In [26]:
## reconstruct a data frame from the logged queries
queries = [literal_eval(q) for q in df['query'].values]
queries = pd.DataFrame(queries)
queries.columns = ['country', 'age', 'subscriber_type', 'num_streams']
print(queries.shape)
queries.head()

(3, 4)


,country,age,subscriber_type,num_streams
0,united_states,24,aavail_basic,8
1,singapore,42,aavail_premium,17
2,united_states,20,aavail_basic,14


In [33]:
from scipy.stats import wasserstein_distance
X_target = pm_tools['preprocessor'].transform(queries)

outlier_test = pm_tools['clf_X'].predict(X_target)
outliers_X = 100 * (1.0 - (outlier_test[outlier_test==1].size / outlier_test.size))
wasserstein_X = wasserstein_distance(pm_tools['X_source'].flatten(),X_target.flatten()) 
wasserstein_y = wasserstein_distance(pm_tools['y_source'],df['y_pred'].values)

if outliers_X >= pm_tools['outlier_X']:
    print("OUTLIER TEST FAILED: {} >= {}".format(round(outliers_X,2),
                                                 pm_tools['outlier_X']))
else:
    print("OUTLIER TEST PASSED: {} < {}".format(round(outliers_X,2),
                                                pm_tools['outlier_X']))
    
if wasserstein_X >= pm_tools['wasserstein_X']:
    print("DISTRIBUTION X TEST FAILED: {} >= {}".format(round(wasserstein_X,2),
                                                        pm_tools['wasserstein_X']))
else:
    print("DISTRIBUTION X TEST PASSED: {} < {}".format(round(wasserstein_X),
                                                       pm_tools['wasserstein_X']))
    
if wasserstein_y >= pm_tools['wasserstein_y']:
    print("DISTRIBUTION y TEST FAILED: {} >= {}".format(round(wasserstein_y,2),
                                                        pm_tools['wasserstein_y']))
else:
    print("DISTRIBUTION y TEST PASSED: {} < {}".format(round(wasserstein_y),
                                                       pm_tools['wasserstein_y']))


OUTLIER TEST PASSED: 0.0 < 2.1
DISTRIBUTION X TEST FAILED: 0.14 >= 0.02
DISTRIBUTION y TEST FAILED: 0.04 >= 0.04
